## Tempering from a normal distribution towards the posterior of a logistic regression
This notebook implements the tempering towards a logistic posterior


In [1]:
import numpy as np
import numexpr as ne
from numba import jit
from matplotlib import pyplot as plt
import sys
sys.path.append("/home/alex/Dropbox/smc_hmc/python_smchmc/")
sys.path.append("/home/alex/Dropbox/smc_hmc/python_smchmc/smc_sampler_functions")
sys.path.append("/home/alex/Dropbox/smc_hmc/python_smchmc/smc_sampler_functions/data")


Specify dimension

In [4]:
dim = 5
import os
os.chdir('/home/alex/Dropbox/smc_hmc/python_smchmc/')

Defining the sampler parameters

In [6]:
N_particles = 2**11
T_time = 20
move_steps_hmc = 20
move_steps_rw_mala = 200
ESStarget = 0.9
M_num_repetions = 10
epsilon = .1
epsilon_hmc = .1
verbose = False
targetmean = np.ones(dim)*2.
targetvariance = (0.1*(np.diag(np.linspace(start=0.01, stop=100, num=dim))/float(dim) +0.7*np.ones((dim, dim))))
targetvariance_inv = np.linalg.inv(targetvariance)
l_targetvariance_inv = np.linalg.cholesky(targetvariance_inv)
factor_variance = 1.
parameters = {'dim' : dim, 
              'N_particles' : N_particles, 
              'factor_variance': factor_variance,
             }

from smc_sampler_functions.functions_smc_help import sequence_distributions
from smc_sampler_functions.proposal_kernels import proposalmala, proposalrw, proposalhmc, proposalhmc_parallel
from smc_sampler_functions.functions_smc_main import smc_sampler
maladict = {'proposalkernel_tune': proposalmala,
                      'proposalkernel_sample': proposalmala,
                      'proposalname' : 'MALA',
                      'target_probability' : 0.65,
                      'covariance_matrix' : np.eye(dim), 
                      'L_max' : 1,
                      'epsilon' : np.array([epsilon]),
                      'epsilon_max' : np.array([epsilon]),
                      'tune_kernel': 'ours_simple',
                      'sample_eps_L' : True,
                      'verbose' : verbose,
                      'move_steps': move_steps_rw_mala,
                      'T_time' : T_time,
                      'autotempering' : True,
                      'ESStarget': ESStarget,
                      'adaptive_covariance' : True,
                      'quantile_test': 0.2
                      }

rwdict = {'proposalkernel_tune': proposalrw,
                      'proposalkernel_sample': proposalrw,
                      'proposalname' : 'RW',
                      'target_probability' : 0.3,
                      'covariance_matrix' : np.eye(dim), 
                      'L_max' : 1,
                      'epsilon' : np.array([epsilon]),
                      'epsilon_max' : np.array([epsilon]),
                      'tune_kernel': 'ours_simple',
                      'sample_eps_L' : True,
                      'verbose' : verbose,
                      'move_steps': move_steps_rw_mala,
                      'T_time' : T_time,
                      'autotempering' : True,
                      'ESStarget': ESStarget,
                      'adaptive_covariance' : True,
                      'quantile_test': 0.2
                      }

hmcdict_adaptive = {'proposalkernel_tune': proposalhmc,
                      'proposalkernel_sample': proposalhmc_parallel,
                      'proposalname' : 'HMC_L_random_ft_adaptive',
                      'target_probability' : 0.9,
                      'covariance_matrix' : np.eye(dim), 
                      'L_max' : 50,
                      'epsilon' : np.array([epsilon_hmc]),
                      'epsilon_max' : np.array([epsilon_hmc]),
                      'accept_reject' : True,
                      'tune_kernel': 'fearnhead_taylor',
                      'sample_eps_L' : True,
                      'parallelize' : False,
                      'verbose' : verbose,
                      'move_steps': move_steps_hmc, 
                      'mean_L' : False,
                      'T_time' : T_time,
                      'autotempering' : True,
                      'ESStarget': ESStarget,
                      'adaptive_covariance' : True,
                      'quantile_test': 0.2
                      }



from smc_sampler_functions.functions_smc_main import repeat_sampling
from smc_sampler_functions.target_distributions import priorlogdens, priorgradlogdens, priorsampler
from smc_sampler_functions.target_distributions import targetlogdens_normal, targetgradlogdens_normal
#from smc_sampler_functions.target_distributions import targetlogdens_student, targetgradlogdens_student
from smc_sampler_functions.target_distributions import targetlogdens_probit, targetgradlogdens_probit, f_dict_logistic_regression
from smc_sampler_functions.target_distributions import targetlogdens_logistic, targetgradlogdens_logistic, f_dict_logistic_regression
parameters_logistic = f_dict_logistic_regression(dim)#, load_mean_var=True, model_type='logistic')
#import ipdb; ipdb.set_trace()
parameters.update(parameters_logistic)


priordistribution = {'logdensity' : priorlogdens, 'gradlogdensity' : priorgradlogdens, 'priorsampler': priorsampler}
#targetdistribution = {'logdensity' : targetlogdens_normal, 'gradlogdensity' : targetgradlogdens_normal, 'target_name': 'normal'}
targetdistribution = {'logdensity' : targetlogdens_logistic, 'gradlogdensity' : targetgradlogdens_logistic, 'target_name': 'logistic'}
#targetdistribution = {'logdensity' : targetlogdens_student, 'gradlogdensity' : targetgradlogdens_student, 'target_name': 'student'}
samplers_list_dict = [hmcdict_adaptive, rwdict, maladict]


temperedist = sequence_distributions(parameters, priordistribution, targetdistribution)







In [7]:
res_repeated_sampling, res_first_iteration = repeat_sampling(samplers_list_dict, temperedist,  parameters, M_num_repetions=M_num_repetions, save_res=False, save_name = '')

repetition 0 of 10
Now runing smc sampler with HMC_L_random_ft_adaptive kernel
Sampler ended at time 23 after 6.94200992584 seconds 

Now runing smc sampler with RW kernel
Sampler ended at time 833 after 24.1329059601 seconds 

Now runing smc sampler with MALA kernel


KeyboardInterrupt: 

In [ ]:
from smc_sampler_functions.functions_smc_plotting import plot_repeated_simulations, plot_results_single_simulation
plot_repeated_simulations(res_repeated_sampling)
plot_results_single_simulation(res_first_iteration)


In [ ]:
print(np.log(res_repeated_sampling['mean_array'].var(axis=1).sum(axis=1)))
res_repeated_sampling['names_samplers']

In [ ]:
res_first_iteration[0]['epsilon_mean']

In [ ]:
from smc_sampler_functions import standard_mh_sampler
parameters_mh = {'dim' : dim, 
              'N_particles' : 50, 
              'factor_variance': factor_variance,
              'T_time' : 2000
             }
import copy
maladict_mh = copy.copy(maladict)
maladict_mh['epsilon'] = np.array([0.3])
res_dict = standard_mh_sampler.parallel_mh_sampler(temperedist, proposalkerneldict=maladict_mh, parameters=parameters_mh)

In [ ]:
print res_dict['acceptance_rate']
res_dict['particles'][:,:,1000:].mean(axis=2).mean(axis=0)-res_repeated_sampling['mean_array'][2,0,:]

In [ ]:
from scipy.optimize import minimize
from functools import partial
partial_target_max = partial(targetlogdens_logistic, parameters=parameters_logistic) 
def partial_target(x):
    return(partial_target_max(x)*-1)
x0 = np.ones((1,dim))*1
print(partial_target_max(x0))
targetlogdens_logistic(x0, parameters_logistic)
res = minimize(partial_target, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
print(res.x)


In [ ]:
it = 0
plt.scatter(x=res_first_iteration[it]['particles_resampled'][:,0], y=res_first_iteration[it]['particles_resampled'][:,1])
plt.show()
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(fit_intercept =  False)
log_reg.fit(parameters_logistic['X_all'], parameters_logistic['y_all'])
log_reg.get_params()
print(log_reg.coef_)
print(res_first_iteration[0]['particles_resampled'].mean(axis=0))
#print(res_first_iteration[1]['particles_resampled'].mean(axis=0))
#print(res_first_iteration[2]['particles_resampled'].mean(axis=0))
#print(np.percentile(res_first_iteration[0]['particles_resampled'], 50., axis=0))

In [ ]:
# import bayes_logistic
import bayes_logistic as bl
p = parameters_logistic['X_all'].shape[1]
X1 = parameters_logistic['X_all']
y1 = parameters_logistic['y_all'][:,0]
w_prior = np.zeros(p)
H_prior = np.diag(np.ones(p))

#----------------------------------------------------------------------------------------
# randomly permute the data

#----------------------------------------------------------------------------------------
# Do a bayesian fit with this random sample
# The default uses a full Hessian matrix and a Newton's conjugate gradient solver
w_posterior, H_posterior = bl.fit_bayes_logistic(y1, X1, w_prior, H_prior)
print w_posterior
#print (res_first_iteration[0]['particles_resampled'].var(axis=0))
print 1./np.diag(H_posterior)

In [ ]:
# sample from the laplace approximation to get the normalizing constant
from scipy.stats import multivariate_normal
w_posterior
L_posterior = np.linalg.cholesky(H_posterior)
H_posterior_inv = np.linalg.inv(H_posterior)
N_samples = 10**6
samples = np.random.multivariate_normal(mean=w_posterior, cov=H_posterior_inv, size=N_samples)
logweight_proposal = multivariate_normal.logpdf(samples, mean=w_posterior, cov=H_posterior_inv)
logweight_target = temperedist.logdensity(samples, temperature=1.)

samples.shape

diff = logweight_target - logweight_proposal
a_max = (diff).max()
logweights = diff - a_max - np.log(np.exp(diff-a_max).sum())
print (((np.exp(logweights)).sum()**2/(np.exp(logweights)**2).sum()))/N_samples
print np.log(np.exp(diff).mean()), sum(res_first_iteration[0]['Z_list'])
res_repeated_sampling['norm_const'].mean(axis=1)

In [ ]:
# importance sampling
(samples*np.exp(diff)[:,np.newaxis]).sum(axis=0)/np.exp(diff).sum(axis=0)
